In [1]:
import pandas as pd

# Load merged Qwen2.5 output
df = pd.read_csv("Qwen2.5_Merged_Multilingual.csv")

# Check if all three predictions agree
df["All_Agree"] = (
    (df["Qwen2.5_EN"] == df["Qwen2.5_FA"]) &
    (df["Qwen2.5_EN"] == df["Qwen2.5_IT"])
)

# Calculate agreement rate
agreement_rate = df["All_Agree"].mean() * 100
print(f"✅ Full agreement across EN, FA, IT: {agreement_rate:.2f}%")


✅ Full agreement across EN, FA, IT: 41.67%


In [2]:
df["Disagree"] = ~df["All_Agree"]
disagreements_by_domain = df[df["Disagree"]].groupby("Domain_EN").size()
print(disagreements_by_domain)

Domain_EN
Activism & Social Change    2
Arts & Culture              4
Business & Economics        3
Caregiving & Parenting      2
Education                   2
Healthcare                  3
Hospitality & Service       4
Journalism & Media          3
Law & Justice               4
Politics & Leadership       3
Religion & Spirituality     3
Science & Technology        2
dtype: int64


In [3]:
import pandas as pd

# Load the merged Qwen2.5 output
df = pd.read_csv("Qwen2.5_Merged_Multilingual.csv")

# Add Disagreement_Type column
df["Disagreement_Type"] = df.apply(lambda row: 
    "FA diverged" if row["Qwen2.5_EN"] == row["Qwen2.5_IT"] != row["Qwen2.5_FA"] else
    "IT diverged" if row["Qwen2.5_EN"] == row["Qwen2.5_FA"] != row["Qwen2.5_IT"] else
    "EN diverged" if row["Qwen2.5_FA"] == row["Qwen2.5_IT"] != row["Qwen2.5_EN"] else
    "All different", axis=1
)

# Print breakdown
print(df["Disagreement_Type"].value_counts())


Disagreement_Type
All different    32
IT diverged      12
FA diverged      11
EN diverged       5
Name: count, dtype: int64


In [4]:
print("Option frequency in FA:")
print(df["Qwen2.5_FA"].value_counts(normalize=True).round(2) * 100)

Option frequency in FA:
Qwen2.5_FA
B    50.0
C    25.0
A    15.0
D    10.0
Name: proportion, dtype: float64


In [5]:
import pandas as pd

# Load your aligned Qwen2.5 results
df = pd.read_csv("Qwen2.5_Merged_Multilingual.csv")

# Identify disagreement rows (not all 3 agree)
df["Disagreement"] = (df["Qwen2.5_EN"] != df["Qwen2.5_FA"]) | \
                     (df["Qwen2.5_EN"] != df["Qwen2.5_IT"]) | \
                     (df["Qwen2.5_FA"] != df["Qwen2.5_IT"])

# Export only rows with disagreement
df_disagreement = df[df["Disagreement"] == True]
df_disagreement.to_csv("Qwen2.5_Disagreements.csv", index=False)
print(f"✅ Exported {len(df_disagreement)} disagreement cases to Qwen2.5_Disagreements.csv")

# Count disagreement cases per domain
disagreement_by_domain = df_disagreement["Domain_EN"].value_counts().reset_index()
disagreement_by_domain.columns = ["Domain", "Disagreement_Count"]
disagreement_by_domain.to_csv("Qwen2.5_Disagreement_By_Domain.csv", index=False)
print("✅ Saved domain-wise disagreement counts to Qwen2.5_Disagreement_By_Domain.csv")

# Show top domains with most disagreement
print("\n📊 Top domains with most disagreement:")
print(disagreement_by_domain.head(10))


✅ Exported 35 disagreement cases to Qwen2.5_Disagreements.csv
✅ Saved domain-wise disagreement counts to Qwen2.5_Disagreement_By_Domain.csv

📊 Top domains with most disagreement:
                    Domain  Disagreement_Count
0           Arts & Culture                   4
1    Hospitality & Service                   4
2            Law & Justice                   4
3    Politics & Leadership                   3
4       Journalism & Media                   3
5  Religion & Spirituality                   3
6               Healthcare                   3
7     Business & Economics                   3
8     Science & Technology                   2
9                Education                   2


In [6]:
import pandas as pd
from collections import Counter

# Load merged results file for Qwen2.5
df = pd.read_csv("Qwen2.5_Merged_Multilingual.csv")

# Check full agreement
df["Full_Agreement"] = (df["Qwen2.5_EN"] == df["Qwen2.5_FA"]) & (df["Qwen2.5_EN"] == df["Qwen2.5_IT"])

# Classify type of disagreement
def classify_disagreement(row):
    choices = {row["Qwen2.5_EN"], row["Qwen2.5_FA"], row["Qwen2.5_IT"]}
    if len(choices) == 1:
        return "All same"
    elif len(choices) == 3:
        return "All different"
    elif row["Qwen2.5_EN"] != row["Qwen2.5_FA"] and row["Qwen2.5_EN"] == row["Qwen2.5_IT"]:
        return "FA diverged"
    elif row["Qwen2.5_EN"] != row["Qwen2.5_IT"] and row["Qwen2.5_EN"] == row["Qwen2.5_FA"]:
        return "IT diverged"
    elif row["Qwen2.5_EN"] != row["Qwen2.5_FA"] and row["Qwen2.5_FA"] == row["Qwen2.5_IT"]:
        return "EN diverged"
    else:
        return "Other"

df["Disagreement_Type"] = df.apply(classify_disagreement, axis=1)

# Count full agreement
agreement_count = df["Full_Agreement"].sum()
total = len(df)
agreement_pct = agreement_count / total * 100

# Disagreement breakdown
disagreement_summary = df["Disagreement_Type"].value_counts().to_frame("count")

# Frequency of choices in Persian
fa_freq = df["Qwen2.5_FA"].value_counts(normalize=True).rename("proportion").to_frame()

# Domain-wise disagreement
domain_disagreements = df[df["Disagreement_Type"] != "All same"]["Domain_EN"].value_counts().to_frame("count")

# Display results
print("🔎 Full Agreement Rate:")
print(f"{agreement_count}/{total} prompts ({agreement_pct:.2f}%)\n")

print("🧩 Disagreement Types:")
print(disagreement_summary, "\n")

print("🗳️ Option Frequency in Persian (Qwen2.5_FA):")
print(fa_freq, "\n")

print("🌍 Domain-wise Disagreements:")
print(domain_disagreements)


🔎 Full Agreement Rate:
25/60 prompts (41.67%)

🧩 Disagreement Types:
                   count
Disagreement_Type       
All same              25
IT diverged           12
FA diverged           11
All different          7
EN diverged            5 

🗳️ Option Frequency in Persian (Qwen2.5_FA):
            proportion
Qwen2.5_FA            
B                 0.50
C                 0.25
A                 0.15
D                 0.10 

🌍 Domain-wise Disagreements:
                          count
Domain_EN                      
Arts & Culture                4
Hospitality & Service         4
Law & Justice                 4
Politics & Leadership         3
Journalism & Media            3
Religion & Spirituality       3
Healthcare                    3
Business & Economics          3
Science & Technology          2
Education                     2
Activism & Social Change      2
Caregiving & Parenting        2


In [1]:
import pandas as pd
import numpy as np

def bootstrap_qwen25_agreement_CI(filepath, prefix="Qwen2.5", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)
    en, fa, it = f"{prefix}_EN", f"{prefix}_FA", f"{prefix}_IT"
    df["Full_Agreement"] = (df[en] == df[fa]) & (df[en] == df[it])

    bootstrapped_agreements = []
    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        agreement_pct = sample["Full_Agreement"].mean() * 100
        bootstrapped_agreements.append(agreement_pct)

    lower_bound = np.percentile(bootstrapped_agreements, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_agreements, (1 + confidence) / 2 * 100)
    mean_agreement = np.mean(bootstrapped_agreements)

    print(f"Bootstrapped {int(confidence*100)}% CI for Qwen2.5 agreement:")
    print(f"Mean agreement: {mean_agreement:.2f}%")
    print(f"{int(confidence*100)}% Confidence Interval: [{lower_bound:.2f}%, {upper_bound:.2f}%]")

# Example usage:
bootstrap_qwen25_agreement_CI("Qwen2.5_Merged_Multilingual.csv")


Bootstrapped 95% CI for Qwen2.5 agreement:
Mean agreement: 41.96%
95% Confidence Interval: [29.96%, 55.00%]


In [3]:
import pandas as pd
import numpy as np

def bootstrap_qwen25_disagreement_CI(filepath, prefix="Qwen2.5", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)
    en, fa, it = f"{prefix}_EN", f"{prefix}_FA", f"{prefix}_IT"
    df["Disagreement"] = ~((df[en] == df[fa]) & (df[en] == df[it]))

    bootstrapped_disagreements = []
    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        disagreement_pct = sample["Disagreement"].mean() * 100
        bootstrapped_disagreements.append(disagreement_pct)

    lower_bound = np.percentile(bootstrapped_disagreements, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_disagreements, (1 + confidence) / 2 * 100)
    mean_disagreement = np.mean(bootstrapped_disagreements)

    print(f"Bootstrapped {int(confidence*100)}% CI for {prefix} disagreement:")
    print(f"Mean disagreement: {mean_disagreement:.2f}%")
    print(f"{int(confidence*100)}% Confidence Interval: [{lower_bound:.2f}%, {upper_bound:.2f}%]")

# Call the function
bootstrap_qwen25_disagreement_CI("Qwen2.5_Merged_Multilingual.csv")


Bootstrapped 95% CI for Qwen2.5 disagreement:
Mean disagreement: 58.57%
95% Confidence Interval: [46.67%, 70.04%]
